In [23]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import time
from collections import Counter
from functools import partial

from sklearn.model_selection import cross_validate, train_test_split, StratifiedShuffleSplit
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn import preprocessing, datasets
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

import mlrose_hiive
from mlrose_hiive import ExpDecay


In [2]:
SEED = 2023
random.seed(SEED)

In [3]:
df = pd.read_csv('wine.csv')

In [4]:
df

,class,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,Od_Hod_of_diluted_wines,Proline
0,1,13.761443,0.972960,2.191146,14.349954,106.525734,2.288580,2.584551,0.418044,1.862804,3.086808,1.169861,2.819086,850.678431
1,2,11.897923,1.297780,2.199666,17.478054,104.205421,2.716657,1.855066,0.318077,1.029640,9.103668,1.353718,2.780333,332.922006
2,3,12.659632,1.748552,2.438597,21.584696,88.079451,1.919476,0.681341,0.538078,1.327974,8.803633,0.643678,1.569951,633.531687
3,1,13.054828,1.817559,1.893926,15.390157,108.589893,2.662509,2.815723,0.278535,1.473812,6.386732,0.820398,3.694865,880.103325
4,3,11.769431,3.017528,3.013448,17.081528,86.525235,2.224963,0.846229,0.286065,1.644007,10.383464,0.734288,2.747013,475.152492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2,13.128402,1.567299,2.760493,24.415990,126.466217,2.616345,1.251803,0.315177,1.853681,6.755482,1.089335,3.345450,860.506822
4996,3,13.444017,3.165880,2.264476,19.261044,114.393216,1.586602,2.700998,0.246732,1.334946,10.688976,0.653591,1.304341,663.953128
4997,2,12.072212,5.173901,2.648389,25.107617,80.672562,2.734035,1.042991,0.392655,1.308256,5.555821,1.237719,3.236121,421.733045
4998,2,12.346772,2.136426,2.493339,21.097520,88.113824,2.643525,2.532831,0.309750,1.894668,3.412186,1.171358,3.025854,301.165911


In [5]:
df.describe()

,class,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,Od_Hod_of_diluted_wines,Proline
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,1.966600,12.995186,2.330916,2.370151,19.496761,99.072699,2.300694,2.057249,0.362036,1.590397,5.086170,0.962753,2.609474,744.635286
std,0.772272,0.814378,1.118433,0.274757,3.351535,13.951364,0.617459,0.992019,0.124728,0.566114,2.337002,0.225385,0.704532,313.348736
min,1.000000,10.991005,0.478367,1.447951,10.366806,68.871553,0.969383,0.277914,0.109327,0.099295,1.129366,0.414259,1.112145,231.883252
25%,1.000000,12.386366,1.592804,2.236354,17.372474,88.827614,1.791985,1.223907,0.270011,1.238714,3.271563,0.802881,1.996074,507.083531
50%,2.000000,13.026910,1.867287,2.369439,19.383204,97.561288,2.324534,2.138558,0.335068,1.546307,4.745563,0.977078,2.770427,665.609642
75%,3.000000,13.624392,2.940638,2.542378,21.429700,105.978061,2.755317,2.808482,0.442366,1.888451,6.214175,1.104621,3.146931,948.204369
max,3.000000,14.802882,6.242546,3.167812,30.236014,164.765854,3.882879,4.397836,0.756410,3.700137,13.120260,1.603007,4.032795,1782.712893


In [6]:
Counter(df['class'])

Counter({1: 1577, 2: 2013, 3: 1410})

In [7]:
df.columns

Index(['class', 'Alcohol', 'Malic_acid', 'Ash', 'Alcalinity_of_ash',
       'Magnesium', 'Total_phenols', 'Flavanoids', 'Nonflavanoid_phenols',
       'Proanthocyanins', 'Color_intensity', 'Hue', 'Od_Hod_of_diluted_wines',
       'Proline'],
      dtype='object')

In [8]:
target = df["class"]
x_col = ['Alcohol', 'Malic_acid', 'Ash', 'Alcalinity_of_ash',
       'Magnesium', 'Total_phenols', 'Flavanoids', 'Nonflavanoid_phenols',
       'Proanthocyanins', 'Color_intensity', 'Hue', 'Od_Hod_of_diluted_wines',
       'Proline']

min_max_scaler = MinMaxScaler()
df_norm = min_max_scaler.fit_transform(df[x_col])
df_norm = pd.DataFrame(df_norm, columns=x_col)
df_norm["class"] = target.values

split = StratifiedShuffleSplit(n_splits=1 , test_size=0.2, random_state=30)
for train_ind, test_ind in split.split(df_norm, df_norm["class"]):
    train = df_norm.loc[train_ind]
    test = df_norm.loc[test_ind]

train_y = train[["class"]]
train_X = train.drop("class", axis=1)
test_y = test[["class"]]
test_X = test.drop("class", axis=1)

## Random Hill Climb

In [24]:
%%time
grid_search_parameters = ({
  'learning_rate': [0.01, 0.001],
  'restarts': [5, 20],
  'activation': [mlrose_hiive.neural.activation.relu]
})
rhc_nn = mlrose_hiive.NNGSRunner(train_X, pd.get_dummies(train_y.values.ravel()).values, 
                                 test_X, pd.get_dummies(test_y.values.ravel()).values, 
                                 "NeuralNetworkRHC",
                                 output_directory="./",
                                 seed=SEED, iteration_list=[1000], 
                                 algorithm=mlrose_hiive.random_hill_climb,
                                 hidden_layer_sizes=[[16,32]],
                                 grid_search_parameters=grid_search_parameters,
                                 grid_search_scorer_method=partial(f1_score, average="weighted"),
                                 n_jobs=-2, cv=5)
result = rhc_nn.run()

Fitting 5 folds for each of 4 candidates, totalling 20 fits
CPU times: user 21.9 s, sys: 473 ms, total: 22.3 s
Wall time: 1min 40s


In [25]:
y_pred_train = result[3].predict(train_X)
y_pred = result[3].predict(test_X)
print('Training set performance: ')
print(classification_report(pd.get_dummies(train_y.values.ravel()).values, y_pred_train))
print('Test set performance: ')
print(classification_report(pd.get_dummies(test_y.values.ravel()).values, y_pred))

Training set performance: 
              precision    recall  f1-score   support

           0       0.23      0.40      0.29      1262
           1       0.48      0.54      0.51      1610
           2       0.00      0.00      0.00      1128

   micro avg       0.34      0.34      0.34      4000
   macro avg       0.24      0.31      0.27      4000
weighted avg       0.27      0.34      0.30      4000
 samples avg       0.34      0.34      0.34      4000

Test set performance: 
              precision    recall  f1-score   support

           0       0.24      0.41      0.30       315
           1       0.50      0.56      0.53       403
           2       0.00      0.00      0.00       282

   micro avg       0.35      0.35      0.36      1000
   macro avg       0.24      0.32      0.28      1000
weighted avg       0.27      0.35      0.31      1000
 samples avg       0.35      0.35      0.35      1000



## Simulated Annealing

In [26]:
%%time
grid_search_parameters = ({
  'schedule': [ExpDecay(1),ExpDecay(10),ExpDecay(100)],
  'learning_rate': [0.0001, 0.001, 0.01],  
  'activation': [mlrose_hiive.neural.activation.relu],
  'max_iters': [1000]
})
sa_nn = mlrose_hiive.NNGSRunner(x_train=train_X,
                     y_train=pd.get_dummies(train_y.values.ravel()).values,
                     x_test=test_X,
                     y_test=pd.get_dummies(test_y.values.ravel()).values,
                     experiment_name='sa',
                     seed=SEED,
                     hidden_layer_sizes=[[16,32]],                             
                     algorithm=mlrose_hiive.algorithms.sa.simulated_annealing,
                     grid_search_parameters=grid_search_parameters,
                     grid_search_scorer_method=partial(f1_score, average="weighted"),
                     iteration_list=[1000],
                     n_jobs=-2)
sa_results = sa_nn.run() 

Fitting 5 folds for each of 9 candidates, totalling 45 fits
CPU times: user 35.3 s, sys: 726 ms, total: 36.1 s
Wall time: 1min 32s


In [27]:
y_pred_train = sa_results[3].predict(train_X)
y_pred = sa_results[3].predict(test_X)
print('Training set performance: ')
print(classification_report(pd.get_dummies(train_y.values.ravel()).values, y_pred_train))
print('Test set performance: ')
print(classification_report(pd.get_dummies(test_y.values.ravel()).values, y_pred))

Training set performance: 
              precision    recall  f1-score   support

           0       0.20      0.32      0.24      1262
           1       0.44      0.54      0.49      1610
           2       0.00      0.00      0.00      1128

   micro avg       0.32      0.32      0.32      4000
   macro avg       0.21      0.29      0.24      4000
weighted avg       0.24      0.32      0.27      4000
 samples avg       0.32      0.32      0.32      4000

Test set performance: 
              precision    recall  f1-score   support

           0       0.20      0.32      0.25       315
           1       0.45      0.55      0.49       403
           2       0.00      0.00      0.00       282

   micro avg       0.32      0.32      0.32      1000
   macro avg       0.22      0.29      0.25      1000
weighted avg       0.24      0.32      0.28      1000
 samples avg       0.32      0.32      0.32      1000



## Genetic Algo

In [28]:
%%time
ga_nn = mlrose_hiive.NeuralNetwork(hidden_nodes = [16,32], activation = 'relu',
                                 algorithm = 'genetic_alg', 
                                 max_iters = 10000,
                                 bias = True, is_classifier = True, 
                                 learning_rate = 0.01,
                                 early_stopping = True,
                                 max_attempts = 100,
                                 random_state = SEED,
                                 pop_size=100,
                                 mutation_prob=0.1)
ga_nn.fit(train_X, pd.get_dummies(train_y.values.ravel()).values)
        

CPU times: user 6min 50s, sys: 7.1 s, total: 6min 58s
Wall time: 1min 46s


NeuralNetwork(algorithm='genetic_alg', early_stopping=True,
              hidden_nodes=[16, 32], learning_rate=0.01, max_attempts=100,
              max_iters=10000, pop_size=100, random_state=2023)

In [29]:
y_pred_train = ga_nn.predict(train_X)
y_pred = ga_nn.predict(test_X)
print('Training set performance: ')
print(classification_report(pd.get_dummies(train_y.values.ravel()).values, y_pred_train))
print('Test set performance: ')
print(classification_report(pd.get_dummies(test_y.values.ravel()).values, y_pred))

Training set performance: 
              precision    recall  f1-score   support

           0       0.75      0.82      0.78      1262
           1       0.80      0.81      0.81      1610
           2       0.84      0.72      0.78      1128

   micro avg       0.79      0.79      0.79      4000
   macro avg       0.79      0.79      0.79      4000
weighted avg       0.79      0.79      0.79      4000
 samples avg       0.79      0.79      0.79      4000

Test set performance: 
              precision    recall  f1-score   support

           0       0.76      0.88      0.82       315
           1       0.84      0.80      0.82       403
           2       0.86      0.76      0.81       282

   micro avg       0.82      0.82      0.82      1000
   macro avg       0.82      0.82      0.81      1000
weighted avg       0.82      0.82      0.82      1000
 samples avg       0.82      0.82      0.82      1000

